In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [3]:
def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w,                       # l1a shape=(?, 48, 48, 32)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 24, 24, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     # l2a shape=(?, 24, 24, 64)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              # l2 shape=(?, 12, 12, 64)
                        strides=[1, 2, 2, 1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)

    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,                     # l3a shape=(?, 12, 12, 128)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],              # l3 shape=(?, 6, 6, 128)
                        strides=[1, 2, 2, 1], padding='SAME')
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])    # reshape to (?, 4608)
    l3 = tf.nn.dropout(l3, p_keep_conv)

    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)

    pyx = tf.matmul(l4, w_o)
    return pyx

In [4]:
arr = []

with open("./binary_cnn_data.csv", "r") as f:
    for line in f:
        arr.append(list(map(int, line.split(','))))


arr = np.array(arr)
arr.shape

(2622, 6914)

In [7]:
X = tf.placeholder("float", [None, 48, 48, 3])
Y = tf.placeholder("float", [None, 2])

w = init_weights([3, 3, 3, 32])       # 3x3x3 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x64 conv, 128 outputs
w4 = init_weights([4608, 625])        # FC 4608 inputs, 625 outputs
w_o = init_weights([625, 2])          # FC 625 inputs, 2 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")

training_size = 1835
image_size = 6912

In [15]:
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.AdamOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for i in range(100):
        np.random.shuffle(arr)
        trX = arr[:training_size, :image_size].reshape((-1, 48, 48, 3))
        trY = arr[:training_size, image_size:]
        
        sess.run(train_op, feed_dict={X: trX, Y: trY,
                                      p_keep_conv: 0.8, p_keep_hidden: 0.5})

        
        teX = arr[training_size:, :image_size].reshape((-1, 48, 48, 3))
        teY = arr[training_size:, image_size:]
        print(i, np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX,
                                                         Y: teY,
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))



0 0.5857687420584498
1 0.6378653113087674
2 0.4358322744599746
3 0.4320203303684879
4 0.4256670902160102
5 0.6188055908513341
6 0.567979669631512
7 0.5705209656925032
8 0.5730622617534943
9 0.6238881829733164
10 0.6734434561626429
11 0.6594663278271918
12 0.6302414231257941
13 0.6111817026683609
14 0.6594663278271918
15 0.6404066073697586
16 0.6874205844980941
17 0.6353240152477764
18 0.6899618805590851
19 0.7052096569250318
20 0.6378653113087674
21 0.7242693773824651
22 0.7166454891994918
23 0.6658195679796697
24 0.6925031766200762
25 0.735705209656925
26 0.7013977128335451
27 0.7217280813214739
28 0.7102922490470139
29 0.6950444726810674
30 0.7407878017789072
31 0.7293519695044473
32 0.7445997458703939
33 0.6759847522236341
34 0.7407878017789072
35 0.7522236340533672
36 0.7407878017789072
37 0.7649301143583227
38 0.7001270648030495
39 0.7255400254129606
40 0.772554002541296
41 0.7738246505717916
42 0.7547649301143583
43 0.7090216010165185
44 0.7623888182973316
45 0.770012706480305
46